In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/PLDAC/

Mounted at /content/gdrive
/content/gdrive/MyDrive/PLDAC


In [252]:
def forget_history(l,k=6,keep_queries=False):
    # amnesia
    mask = np.zeros(len(l),dtype=bool)
    mask[:2] = True
    mask[-k:] = True
    if keep_queries:
        # half amnesia
        mask[np.arange(0,len(mask),2)] = True
    return np.array(l)[mask]

In [246]:
def to_csv(df,dest):
    df.rename(columns={"Rewrite":"Target"},inplace=True)
    df = pd.DataFrame(df[["Question","History","Target"]])
    df["Source"] = "Query: " + df["Question"] + " |||| Context: " + df["History"] + " |||| Reformulation: "
    df.drop(columns=["Question","History"], inplace=True)
    df.to_csv(dest)

In [253]:
df_canard_dev = pd.read_json('./data/canard/dev.json').drop(columns=["QuAC_dialog_id","Question_no"])
df_canard_dev_amnesia = df_canard_dev.copy()
df_canard_dev_half_amnesia = df_canard_dev.copy()

df_canard_dev["History"] = df_canard_dev["History"].apply(lambda x: " ||| ".join(x))
df_canard_dev_amnesia["History"] = df_canard_dev_amnesia["History"].apply(lambda x: " ||| ".join(forget_history(x)))
df_canard_dev_half_amnesia["History"] = df_canard_dev_half_amnesia["History"].apply(lambda x: " ||| ".join(forget_history(x,keep_queries=True)))

In [254]:
df_canard_train = pd.read_json('./data/canard/train.json').drop(columns=["QuAC_dialog_id","Question_no"])
df_canard_train_amnesia = df_canard_train.copy()
df_canard_train_half_amnesia = df_canard_train.copy()

df_canard_train["History"] = df_canard_train["History"].apply(lambda x: " ||| ".join(x))
df_canard_train_amnesia["History"] = df_canard_train_amnesia["History"].apply(lambda x: " ||| ".join(forget_history(x)))
df_canard_train_half_amnesia["History"] = df_canard_train_half_amnesia["History"].apply(lambda x: " ||| ".join(forget_history(x,keep_queries=True)))

In [255]:
df_canard_test = pd.read_json('./data/canard/test.json').drop(columns=["QuAC_dialog_id","Question_no"])
df_canard_test_amnesia = df_canard_test.copy()
df_canard_test_half_amnesia = df_canard_test.copy()

df_canard_test["History"] = df_canard_test["History"].apply(lambda x: " ||| ".join(x))
df_canard_test_amnesia["History"] = df_canard_test_amnesia["History"].apply(lambda x: " ||| ".join(forget_history(x)))
df_canard_test_half_amnesia["History"] = df_canard_test_half_amnesia["History"].apply(lambda x: " ||| ".join(forget_history(x,keep_queries=True)))

In [256]:
to_csv(df_canard_train,"./data/canard/train.csv")
to_csv(df_canard_dev,"./data/canard/dev.csv")
to_csv(df_canard_test,"./data/canard/test.csv")

df_canard_train = pd.read_csv("./data/canard/train.csv")
df_canard_dev = pd.read_csv("./data/canard/dev.csv")
df_canard_test = pd.read_csv("./data/canard/test.csv")

In [248]:
df_canard_test.head()

,Unnamed: 0,Target,Source
0,0,Did investigators have any clues in the unreso...,Query: Did they have any clues? |||| Context: ...
1,1,How did FSB target the murdered journalist Ann...,Query: How did they target her email? |||| Con...
2,2,Did FSB get into trouble for attacking the ema...,Query: Did they get into trouble for that? |||...
3,3,Did FSB have any Anna Politkovskaya's murder s...,Query: Did they have any murder suspects? ||||...
4,4,Did any of Anna Politkovskaya's murder suspect...,Query: Did they go to jail? |||| Context: Anna...


In [257]:
to_csv(df_canard_train_amnesia,"./data/canard/train_amnesia.csv")
to_csv(df_canard_dev_amnesia,"./data/canard/dev_amnesia.csv")
to_csv(df_canard_test_amnesia,"./data/canard/test_amnesia.csv")
df_canard_train_amnesia = pd.read_csv("./data/canard/train_amnesia.csv")
df_canard_dev_amnesia = pd.read_csv("./data/canard/dev_amnesia.csv")
df_canard_test_amnesia = pd.read_csv("./data/canard/test_amnesia.csv")

In [250]:
df_canard_test_amnesia.head()

,Unnamed: 0,Target,Source
0,0,Did investigators have any clues in the unreso...,Query: Did they have any clues? |||| Context: ...
1,1,How did FSB target the murdered journalist Ann...,Query: How did they target her email? |||| Con...
2,2,Did FSB get into trouble for attacking the ema...,Query: Did they get into trouble for that? |||...
3,3,Did FSB have any Anna Politkovskaya's murder s...,Query: Did they have any murder suspects? ||||...
4,4,Did any of Anna Politkovskaya's murder suspect...,Query: Did they go to jail? |||| Context: Anna...


In [258]:
to_csv(df_canard_train_half_amnesia,"./data/canard/train_half_amnesia.csv")
to_csv(df_canard_dev_half_amnesia,"./data/canard/dev_half_amnesia.csv")
to_csv(df_canard_test_half_amnesia,"./data/canard/test_half_amnesia.csv")
df_canard_train_half_amnesia = pd.read_csv("./data/canard/train_half_amnesia.csv")
df_canard_dev_half_amnesia = pd.read_csv("./data/canard/dev_half_amnesia.csv")
df_canard_test_half_amnesia = pd.read_csv("./data/canard/test_half_amnesia.csv")

In [259]:
df_canard_test_half_amnesia.head()

,Unnamed: 0,Target,Source
0,0,Did investigators have any clues in the unreso...,Query: Did they have any clues? |||| Context: ...
1,1,How did FSB target the murdered journalist Ann...,Query: How did they target her email? |||| Con...
2,2,Did FSB get into trouble for attacking the ema...,Query: Did they get into trouble for that? |||...
3,3,Did FSB have any Anna Politkovskaya's murder s...,Query: Did they have any murder suspects? ||||...
4,4,Did any of Anna Politkovskaya's murder suspect...,Query: Did they go to jail? |||| Context: Anna...


In [47]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip

--2022-02-23 20:22:30--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  48.0MB/s    in 14s     

2022-02-23 20:22:44 (48.0 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]



In [ ]:
#!unzip /content/gdrive/MyDrive/PLDAC/wiki-news-300d-1M.vec.zip

Archive:  /content/gdrive/MyDrive/PLDAC/wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [48]:
import io
from collections import defaultdict

def default_vector():
    return np.zeros(300)

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = defaultdict(default_vector)
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(tokens[1:]).astype(float)
    return data

In [ ]:
#!head -n 200000 "/content/gdrive/MyDrive/PLDAC/wiki-news-300d-1M.vec" > "/content/gdrive/MyDrive/PLDAC/w2v_200000.vec"

In [49]:
!head -n 2 "/content/gdrive/MyDrive/PLDAC/w2v_200000.vec"

999994 300
, 0.1073 0.0089 0.0006 0.0055 -0.0646 -0.0600 0.0450 -0.0133 -0.0357 0.0430 -0.0356 -0.0032 0.0073 -0.0001 0.0258 -0.0166 0.0075 0.0686 0.0392 0.0753 0.0115 -0.0087 0.0421 0.0265 -0.0601 0.2420 0.0199 -0.0739 -0.0031 -0.0263 -0.0062 0.0168 -0.0357 -0.0249 0.0190 -0.0184 -0.0537 0.1420 0.0600 0.0226 -0.0038 -0.0675 -0.0036 -0.0080 0.0570 0.0208 0.0223 -0.0256 -0.0153 0.0022 -0.0482 0.0131 -0.6016 -0.0088 0.0106 0.0229 0.0336 0.0071 0.0887 0.0237 -0.0290 -0.0405 -0.0125 0.0147 0.0475 0.0647 0.0474 0.0199 0.0408 0.0322 0.0036 0.0350 -0.0723 -0.0305 0.0184 -0.0026 0.0240 -0.0160 -0.0308 0.0434 0.0147 -0.0457 -0.0267 -0.1703 -0.0099 0.0417 0.0235 -0.0260 -0.1519 -0.0116 -0.0306 -0.0413 0.0330 0.0723 0.0365 -0.0001 0.0042 0.0346 0.0277 -0.0305 0.0784 -0.0404 0.0187 -0.0225 -0.0206 -0.0179 -0.2428 0.0669 0.0523 0.0527 0.0149 -0.0708 -0.0987 0.0263 -0.0611 0.0302 0.0216 0.0313 -0.0140 -0.2495 -0.0346 -0.0480 0.0250 0.2130 -0.0330 -0.1553 -0.0292 -0.0346 0.1074 0.0010 -0.0117 -0.0057

In [50]:
w2v = load_vectors("/content/gdrive/MyDrive/PLDAC/w2v_200000.vec")

In [ ]:
#!pip install sentence-transformers
import spacy
from sentence_transformers import SentenceTransformer, util

def custom_sentencizer(doc):
    ''' Look for sentence start tokens by scanning for periods only. '''
    for i, token in enumerate(doc[:-2]):
        if token.text == ".":
            doc[i+1].is_sent_start = True
        else:
            doc[i+1].is_sent_start = False
    return doc

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe(custom_sentencizer, before="parser")

def sentence_segmentation(passage):
    return [s.text for s in nlp(passage).sents]
    
model = SentenceTransformer('sentence-transformers/msmarco-distilbert-cos-v5')

stemmer = SnowballStemmer("english")
key_tags = set(["CD","FW","JJ","JJR","JJS","NN","NNS","NNP","NNPS","VB","VBD","VBG","VBN","VBP","VBZ"])

In [243]:
def extract(s,stem=False,lower=True):
    # (to lower), tokenize, then filter by tag and remove stopword, (then stem)
    if lower:
        s = s.lower()
    if stem:
        return [stemmer.stem(w) for (w,tag) in nltk.pos_tag(nltk.word_tokenize(s)) if tag in key_tags and w not in stopwords.words('english')]
    else:
        return [w for (w,tag) in nltk.pos_tag(nltk.word_tokenize(s)) if tag in key_tags and w not in stopwords.words('english')]

def sim_overlap(u,ss):
    # number of keywords in common
    return [len(set(extract(s,True,False)) & set(extract(u,True,False))) for s in ss]

def sim_maxsim(u,ss,threshold=1.5):
    # my idea
    sim = np.array([ np.mean([\
                        np.max([\
                            np.dot(w2v[vu],w2v[vs])\
                        for vs in set(extract(s))]+[0])\
                   for vu in set(extract(u))]+[0])\
                for s in ss])
    return sim - threshold
    
def sim_bert(u,ps,threshold=0.3):
    sim = util.dot_score(model.encode(u),model.encode(ps)).numpy()[0]
    return sim - threshold

def key_sentence(p,u_next,u,sim):
    # sentence with highest similarity. Note: argmax favors early sentences in case of hesitation, which is a desirable behaviour.
    
    ss = sentence_segmentation(p)
    if len(ss) == 0:
        return ""
    scores = sim(u_next,ss)
    if np.max(scores,initial=-1) > 0:
        return ss[np.argmax(scores)]
    else:
        scores = sim(u,ss)
        if np.max(scores,initial=-1) > 0:
            return ss[np.argmax(scores)]
    return ""


In [79]:
import json

with open("./data/treccast/treccastweb-master/2021/2021_manual_evaluation_topics_v1.0.json") as json_file:
    data_2021 = json.load(json_file)

In [228]:
class TRECSet():
    def __init__(self, data, year):
        self.topics = [Topic(topic) for topic in data]
        #self.write_history_simple()
        #self.write_history_overlap_extraction()
        #self.write_history_word2vec_extraction()
        #self.write_history_bert_extraction()
        self.year = year
    
    def to_csv(self):
        self.df = pd.concat([topic.to_df() for topic in self.topics])
        for i in range(1,14):
            df_i = pd.DataFrame(self.df[["Question",f"History{i}","Target"]])
            df_i["Source"] = df_i.apply(lambda x: "Query: " + x["Question"] + " |||| Context: " + str(x[f"History{i}"]) + " |||| Reformulation: ", axis=1)
            df_i.drop(columns=["Question",f"History{i}"], inplace=True)
            df_i.to_csv(f"./data/treccast/treccastweb-master/{self.year}/trec{self.year}_{i}.csv")
        self.df.to_csv(f"./data/treccast/treccastweb-master/{self.year}/trec{self.year}.csv")
    
    def write_history_simple(self):
        for topic in self.topics:
            topic.write_history_simple()

    def write_history_overlap_extraction(self):
        for topic in self.topics:
            topic.write_history_overlap_extraction()

    def write_history_word2vec_extraction(self):
        for topic in self.topics:
            topic.write_history_word2vec_extraction()

    def write_history_bert_extraction(self):
        for topic in self.topics:
            topic.write_history_bert_extraction()

class Topic():
    def __init__(self, topic):
        self.number = topic["number"]
        self.turns = [Turn(turn) for turn in topic["turn"]]
    
    def to_df(self):
        return pd.DataFrame([turn.to_array() for turn in self.turns],\
                columns=["Question","Target","History1","History2","History3",\
                "History4","History5","History6","History7","History8",\
                "History9","History10","History11","History12","History13"])

    def write_history_simple(self):
        history1 = []
        history2 = []
        history3 = []
        history4 = []
        for i in range(len(self.turns)):
            # 1> query only
            self.turns[i].history1 = " ||| ".join(history1)
            history1.append(self.turns[i].raw_utterance)
            # 2> recursive query only
            self.turns[i].history2 = " ||| ".join(history2)
            history2.append(self.turns[i].manual_rewritten_utterance)
            # 3> naive
            self.turns[i].history3 = " ||| ".join(history3)
            history3.append(self.turns[i].raw_utterance)
            history3.append(self.turns[i].passage)
            # 4> recursive, top 2 summary
            self.turns[i].history4 = " ||| ".join(history4)
            history4.append(self.turns[i].manual_rewritten_utterance)
            history4.append(" ".join(sentence_segmentation(self.turns[i].passage)[:2]))

    def write_history_overlap_extraction(self):
        history5 = []
        history6 = []
        history7 = []

        for i in range(len(self.turns)):
            # key sentence overlap extraction
            overlap_extracted_sentence = key_sentence(self.turns[i].passage,\
                self.turns[i+1].raw_utterance if i+1<len(self.turns) else "",\
                self.turns[i].raw_utterance, sim=sim_overlap)

            # 5> waterloo (recursive, overlap key sentence extraction, type b)
            self.turns[i].history5 = " ||| ".join(history5)
            if (i-2)*2+1 >= 0:
                history5[(i-2)*2+1] = ""
            history5.append(self.turns[i].manual_rewritten_utterance)
            history5.append(overlap_extracted_sentence)
            
            # 6> recursive, overlap key sentence extraction
            self.turns[i].history6 = " ||| ".join(history6)
            history6.append(self.turns[i].manual_rewritten_utterance)
            history6.append(overlap_extracted_sentence)

            # 7> recursive, overlap key sentence extraction, last 3 paragraphs
            self.turns[i].history7 = " ||| ".join(history7)
            if (i-4)*2+1 > 0:
                history7[(i-4)*2+1] = ""
            history7.append(self.turns[i].manual_rewritten_utterance)
            history7.append(overlap_extracted_sentence)

    def write_history_word2vec_extraction(self):
        history8 = []
        history9 = []
        history10 = []
        for i in range(len(self.turns)):
            # key sentence word2vec maxsim extraction
            maxsim_extracted_sentence = key_sentence(self.turns[i].passage,\
                self.turns[i+1].raw_utterance if i+1<len(self.turns) else "",\
                self.turns[i].manual_rewritten_utterance, sim=sim_maxsim)
            
            # 8> maxsim key sentence extraction, last paragraph
            self.turns[i].history8 = " ||| ".join(history8)
            if (i-2)*2+1 >= 0:
                history8[(i-2)*2+1] = ""
            history8.append(self.turns[i].manual_rewritten_utterance)
            history8.append(maxsim_extracted_sentence)
            
            # 9> maxsim key sentence extraction, all paragraph
            self.turns[i].history9 = " ||| ".join(history9)
            history9.append(self.turns[i].manual_rewritten_utterance)
            history9.append(maxsim_extracted_sentence)

            # 10> maxsim key sentence extraction, 3 last paragraph
            self.turns[i].history10 = " ||| ".join(history10)
            if (i-4)*2+1 >= 0:
                history10[(i-4)*2+1] = ""
            history10.append(self.turns[i].manual_rewritten_utterance)
            history10.append(maxsim_extracted_sentence)


    def write_history_bert_extraction(self):
        history11 = []
        history12 = []
        history13 = []
        for i in range(len(self.turns)):
            # key sentence bert sim extraction
            bert_extracted_sentence = key_sentence(self.turns[i].passage,\
                self.turns[i+1].raw_utterance if i+1<len(self.turns) else "",\
                self.turns[i].manual_rewritten_utterance, sim=sim_bert)
            
            # 11> bert key sentence extraction, last paragraph
            self.turns[i].history11 = " ||| ".join(history11)
            if (i-2)*2+1 >= 0:
                history11[(i-2)*2+1] = ""
            history11.append(self.turns[i].manual_rewritten_utterance)
            history11.append(bert_extracted_sentence)
            
            # 12> bert key sentence extraction, all paragraph
            self.turns[i].history12 = " ||| ".join(history12)
            history12.append(self.turns[i].manual_rewritten_utterance)
            history12.append(bert_extracted_sentence)

            # 13> bert key sentence extraction, 3 last paragraph
            self.turns[i].history13 = " ||| ".join(history13)
            if (i-4)*2+1 >= 0:
                history13[(i-4)*2+1] = ""
            history13.append(self.turns[i].manual_rewritten_utterance)
            history13.append(bert_extracted_sentence)

class Turn():
    def __init__(self, turn):
        self.number = turn["number"]
        self.raw_utterance = turn["raw_utterance"]
        self.passage = turn["passage"]
        self.manual_rewritten_utterance = turn["manual_rewritten_utterance"]
        self.manual_query_history = []
        self.raw_query_history = []
        self.passage_history = []

    def to_array(self):
        return [self.raw_utterance, self.manual_rewritten_utterance,\
                self.history1, self.history2, self.history3, self.history4,\
                self.history5, self.history6, self.history7, self.history8,\
                self.history9, self.history10, self.history11, self.history12,\
                self.history13]

In [191]:
treccast_2021 = TRECSet(data_2021,2021)

In [192]:
treccast_2021.write_history_simple()

In [193]:
treccast_2021.write_history_overlap_extraction()

In [194]:
treccast_2021.write_history_word2vec_extraction()

In [195]:
treccast_2021.write_history_bert_extraction()

In [196]:
treccast_2021.to_csv()

In [ ]:
df_2021 = pd.read_csv("./data/treccast/treccastweb-master/2021/trec2021.csv")
df_2021.head()

In [198]:
import json

with open("/content/gdrive/MyDrive/PLDAC/data/treccast/treccastweb-master/2020/2020_manual_evaluation_topics_v1.0.json") as json_file:
    data_2020 = json.load(json_file)

In [220]:
passages_2020 = dict()
with open("/content/gdrive/MyDrive/PLDAC/mini_corpus/MARCO/marco_passage_2020.tsv","r") as infile:
    lines = infile.readlines()
    for line in lines:
        tmp = line.strip().split(None, 1)
        if len(tmp) > 0:
            id, text = tmp
            passages_2020[id] = text

In [230]:
for topic in data_2020:
    for turn in topic['turn']:
        passage_id = turn['manual_canonical_result_id']
        if passage_id[:6] == "MARCO_":
            turn['passage'] = passages_2020[passage_id[6:]]
        else:
            # I don't have the CAR dataset
            turn['passage'] = ""

In [ ]:
data_2020

In [244]:
treccast_2020 = TRECSet(data_2020,2020)
treccast_2020.write_history_simple()
treccast_2020.write_history_overlap_extraction()
treccast_2020.write_history_word2vec_extraction()
treccast_2020.write_history_bert_extraction()
treccast_2020.to_csv()

In [245]:
df_2020 = pd.read_csv("./data/treccast/treccastweb-master/2020/trec2020.csv")
df_2020.head()

,Unnamed: 0,Question,Target,History1,History2,History3,History4,History5,History6,History7,History8,History9,History10,History11,History12,History13
0,0,How do you know when your garage door opener i...,How do you know when your garage door opener i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Now it stopped working. Why?,Now my garage door opener stopped working. Why?,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...
2,2,How much does it cost for someone to fix it?,How much does it cost for someone to repair a ...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...
3,3,How about replacing it instead?,How much does it cost to replace a garage door...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...
4,4,How do I choose a new one?,How do I choose a new garage door opener?,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...,How do you know when your garage door opener i...
